In [5]:
# Download the dataset to __dataset__ folder.
# http://fsoco.cs.uni-freiburg.de/datasets/fsoco_segmentation_train.zip


def download_dataset(url: str, destination_folder: str):
    import os
    import requests
    import zipfile

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # If meta.json exists, then the dataset has been downloaded.
    if os.path.exists(os.path.join(destination_folder, "meta.json")):
        print("Dataset has been downloaded.")
        return

    filename = url.split("/")[-1]
    filepath = os.path.join(destination_folder, filename)

    if not os.path.exists(filepath):
        print("Downloading dataset...")
        r = requests.get(url, stream=True)
        with open(filepath, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)

    print("Extracting dataset...")
    with zipfile.ZipFile(filepath) as zf:
        zf.extractall(destination_folder)

    print("Done.")

    # Remove the zip file.
    os.remove(filepath)


download_dataset(
    "http://fsoco.cs.uni-freiburg.de/datasets/fsoco_boxes_train.zip",
    "__dataset__/boxes/supervisely",
)

Dataset has been downloaded.


In [6]:
from supervisely_yolo_multiple import S2Y
import os
import glob


# Convert the dataset to YOLO format.
def convert_to_yolo(source_folder: str, destination_folder: str):


    # Create data.yaml file.
    if os.path.exists(destination_folder):
        print("YOLO structure already exists.")
        return

    converter = S2Y(source_folder, destination_folder, skip_copy=True)
    class_names_array = converter.get_class_names_from_supervisely()
    converter.create_yolo_file_structure()
    converter.create_class_file(class_names_array)

    dataset_folders = [
        folder for folder in os.listdir(converter.supervisely_path)
    ]

    for folder in dataset_folders:
        labels_path = converter.supervisely_path + '/' + folder + "//ann"
        print("labels_path", labels_path)
        for file_path in glob.glob(os.path.join(labels_path, "*.json")):
            with open(file_path) as file:
                file_name = os.path.basename(file.name)[:-5]
                converter.create_text_file(folder, file_name, class_names_array)
        print("Yolo structure created at => {}".format(converter.yolo_path))


convert_to_yolo("__dataset__/boxes/supervisely", "__dataset__/boxes/yolo")

YOLO structure already exists.


In [8]:
# Create the data.yaml file and split into train, test and validate.

if os.path.exists("__dataset__/boxes/yolo/data.yaml"):
    print("data.yaml already exists.")
    exit()

# Read the classes from the supervisely folder.
import json
supervisely_meta = json.load(open("__dataset__/boxes/supervisely/meta.json"))
classes = map(lambda x: x["title"], supervisely_meta["classes"])

# Split the dataset into train, test and validate.
import random
import shutil

# Create the train, test and validate folders.
folders = ["train", "test", "validate"]
for folder in folders:
    if not os.path.exists("__dataset__/boxes/yolo/" + folder):
        os.makedirs("__dataset__/boxes/yolo/" + folder)
        os.makedirs("__dataset__/boxes/yolo/" + folder + "/images")
        os.makedirs("__dataset__/boxes/yolo/" + folder + "/labels")

# Copy the images and labels to the train, test and validate folders.
for file in os.listdir("__dataset__/boxes/yolo/images"):

    # Take a random number between 0 and 2.
    random_number = random.randint(0, 2)
    destination_folder = "__dataset__/boxes/yolo/" + folders[random_number]

    # Copy the image and the label to the destination folder.
    shutil.move("__dataset__/boxes/yolo/images/" + file, destination_folder + "/images/" + file)
    file_name_without_extension = os.path.splitext(file)[0]
    shutil.move("__dataset__/boxes/yolo/labels/" + file_name_without_extension + ".txt", destination_folder + "/labels/" + file_name_without_extension + ".txt")

# Create the data.yaml file.
data = {
    'names': classes.tolist(),
    'nc': len(classes),
    'test': '__dataset__/boxes/yolo/test/images',
    'train': '__dataset__/boxes/yolo/train/images',
    'val': '__dataset__/boxes/yolo/validate/images'
}

FileNotFoundError: [Errno 2] No such file or directory: '__dataset__/boxes/yolo/labels/amz_02155.txt'